# HW #2 Julia's Type System

One of the big advantages of Julia for scientific computing is Julia's novel type system.  Historically, practitioners of scientific computing have been ignorant of type systems.  Conversely, experts on type systems have been historically ignorant of scientific computing. The result has been that before Julia, the benefits of a type system have been available in only limited cases and not widely understood.

Along the way we'll learn elements of Julia's syntax that are not obvious to the newcomer but are really useful.
We hope by the end of this homework, the uses of all sorts of "dots"  `.`, `...`,`:`,`::`  will be clear.
(One might refer to the <a href="https://docs.julialang.org/en/v1/base/punctuation/"> Punctuation doc </a>
which can be really helpful.)  Dots in a language are great once you understand them as they use very little "ink" so they don't distract, but if they are mysterious, then the language just seems unreadable.

The dot `.` is used for decimal points, field names, and broadcast. <br>
The triple dots `...` is used for slurping and splatting. (Love those names.) <br>
The `:` is used for ranges, in indexing, and to create a symbol (offically known as a quoted expression). <br>
The `::` is used for type annotation.



Jupyter Tip: Esc + A and Esc + B (insert cell above/below -- teach your fingers this and you'll thank your fingers)

**Warning**: Structs can't be redefined.  You'll have to reload the kernel and re-execute.

# Built in types in Julia

Some useful commands are <a  href="https://docs.julialang.org/en/v1/base/base/#Core.typeof"> typeof </a>,
<a href="https://docs.julialang.org/en/v1/base/base/#Core.isa"> isa </a>, 
<a href="https://docs.julialang.org/en/v1/base/io-network/#Base.dump"> dump </a>, 
and <a href="https://docs.julialang.org/en/v1/base/numbers/#Base.bitstring"> bitstring </a>.

1. Create a "vector"  `v` of type `Array{Int64,1}` which has at least two positive and two negative integers.

In [2]:
v = [4, 15, -12, 0, -1, -0]

6-element Array{Int64,1}:
   4
  15
 -12
   0
  -1
   0

2. Run the `bitstring` command on each element of v and interpret with precision the bits that you see.

**Interpretation**:
* 4 and 15 are easy as 4 is 100 in base 2 and 15 is 1111 in base 2.
* As 0 and -0 are just 0 in base 2, we can see that Julia does not use represent negative integers using one's complement.
* Negative numbers are represented using two's complement.

In [88]:
bitstring.(v)

7-element Array{String,1}:
 "0100000000000100000000000000000000000000000000000000000000000000"
 "0100000001000111110011001100110011001100110011001100110011001101"
 "1011111111000010010010010010010010010010010010010010010010010010"
 "0000000000000000000000000000000000000000000000000000000000000000"
 "1111110111011101110011110001110011001101111101111011110000101101"
 "0000000000000000000000000000000000000000000000000000000000000000"
 "0011100100001011100001011111100011000101010001000101111100000010"

3. Create a "vector" `v` of type `Array{Float64,1}` which has at least two positive and two negative values. Including a few that are not exactly mathematical integers.

In [10]:
v = [5/2, 47.6, -1/7, 0, -π^600, -0, 6.626e-34]

7-element Array{Float64,1}:
  2.5                   
 47.6                   
 -0.14285714285714285   
  0.0                   
 -1.9495016921159666e298
  0.0                   
  6.626e-34             

4. Interpret the bitstring for floats. Please read and digest the wikipedia article on the <a href="https://en.wikipedia.org/wiki/Double-precision_floating-point_format#IEEE_754_double-precision_binary_floating-point_format:_binary64">IEEE 754 double-precision binary floating-point format </a>.

**Interpretation**:
* The first bit stores the sign (0 for +, 1 for -) so we see that -1/7 and -π^600 start with a 1.
* The next 11 bits store a number e from 0 to 2048 to calculate the exponent as e-1023. Thus 2.5, 47.6, and -1/7 have exponents close to 1024, while -π^600 has a number larger than 1024 (it's 2013) and 6.626e-34 is smaller than 1024 (it's 912).
* The next 52 bits store the fraction. We see that only 1 bit is required to represent the fraction 0.5 in 2.5, a regular pattern represents the fraction in 47.6 and -1/7, while the fractions in -π^600 and 6.626e-34 are irregular.
* 0 and -0 have the same representation as zero is signed.

In [12]:
bitstring.(v)

7-element Array{String,1}:
 "0100000000000100000000000000000000000000000000000000000000000000"
 "0100000001000111110011001100110011001100110011001100110011001101"
 "1011111111000010010010010010010010010010010010010010010010010010"
 "0000000000000000000000000000000000000000000000000000000000000000"
 "1111110111011101110011110001110011001101111101111011110000101101"
 "0000000000000000000000000000000000000000000000000000000000000000"
 "0011100100001011100001011111100011000101010001000101111100000010"

5. The machine learning world is more interested in approximate than exact answers has motivated increased use of half precision (float16, binary16).  This can save time and memory. Explain the format of half precision for a few numbers such as 1.0.  <a href="https://en.wikipedia.org/wiki/Half-precision_floating-point_format#IEEE_754_half-precision_binary_floating-point_format:_binary16"> Wikipedia article on half precision. </a>

**Explanation**: Float16 is basically the same as Float64 described above except that the exponent is 5 bits with a 10 bit fraction.

In [6]:
bitstring(Float16(1.0))

"0011110000000000"

# Composite Types

Let's create a function that takes an Int as input and returns a vector containing an Int64, Float64, Float16, and String.

In [89]:
function vector_of_types(n::Int)
    [Int64(n), Float64(n), Float16(n), string(n)]
end

vector_of_types (generic function with 1 method)

In [14]:
vector_of_types(17)

4-element Array{Any,1}:
         17    
         17.0  
 Float16(17.0) 
           "17"

Notice this has type `Array{Any,1}`

In [15]:
typeof(vector_of_types(17))

Array{Any,1}

Let's now create a composite type

In [16]:
struct FourTypes
    n::Int64
    x::Float64
    y::Float16
    z::String
end

Create an `f` of this type using a <a href="https://docs.julialang.org/en/v1/manual/constructors/#"> constructor. </a>

In [17]:
f = FourTypes(17,17.0,Float16(17),"17")  

FourTypes(17, 17.0, Float16(17.0), "17")

6. Why is the following properly an error?

**Answer**: The constructor was expecting the fourth argument to be of type String.

In [18]:
f = FourTypes(17,17,17,17)

MethodError: MethodError: Cannot `convert` an object of type Int64 to an object of type String
Closest candidates are:
  convert(::Type{T<:AbstractString}, !Matched::T<:AbstractString) where T<:AbstractString at strings/basic.jl:207
  convert(::Type{T<:AbstractString}, !Matched::AbstractString) where T<:AbstractString at strings/basic.jl:208
  convert(::Type{T}, !Matched::T) where T at essentials.jl:154

I love the <a href="https://docs.julialang.org/en/v0.6.1/manual/faq/#The-two-uses-of-the-...-operator:-slurping-and-splatting-1"> splat </a> operator.  Here's an example.

In [19]:
f = FourTypes(vector_of_types(17)...) 

FourTypes(17, 17.0, Float16(17.0), "17")

In [20]:
dump(f)

FourTypes
  n: Int64 17
  x: Float64 17.0
  y: Float16 Float16(17.0)
  z: String "17"


In [21]:
typeof(f)

FourTypes

6. Create a new type called `FourVectorTypes` with field `vn`,`vx`,`vy`,`vz` which contains respectively vectors of int64s, float64s, float16s, strings

In [22]:
struct FourVectorTypes
    vn::Vector{Int64}
    vx::Vector{Float64}
    vy::Vector{Float16}
    vz::Vector{String}
end

In [23]:
## Test your type
FourVectorTypes([1,2,3],[1.0,2,3],Float16.([1,2,3]),["1","2","3","4"])

FourVectorTypes([1, 2, 3], [1.0, 2.0, 3.0], Float16[1.0, 2.0, 3.0], ["1", "2", "3", "4"])

7. play with <a href="https://docs.julialang.org/en/v1/base/base/#Base.isbits">isbits</a> and
<a href="https://docs.julialang.org/en/v1/base/base/#Base.sizeof-Tuple{Type}">sizeof </a> and tell us what
you learn for some built-in types and composite types.  Try a type with an Int64 and a Float32.  Any surprise?

**No surprises.** sizeof returns the sizes as expected, and spits an error for types of indeterminate sizes such as strings and arrays. isbits returns true for immutable basic types and false otherwise.

In [39]:
sizeof.([Float16, Float64, Int8, Int32, UInt128, Complex{Float64}, Complex{Int8}, Complex{Int128}])

8-element Array{Int64,1}:
  2
  8
  1
  4
 16
 16
  2
 32

In [40]:
isbits.([1, -2.0, 1/7, -1//13, Complex(4, -1), "ABC", [1,4,4]])

7-element BitArray{1}:
  true
  true
  true
  true
  true
 false
 false

# ... but Julia seems to be doing something more interesting or what are all those curly braces and that crazy letter "T" we keep seeing?

In [41]:
struct MyTypeAndMe{T}
    n::T
end

In [42]:
MyTypeAndMe(17)

MyTypeAndMe{Int64}(17)

In [43]:
typeof(MyTypeAndMe(17))

MyTypeAndMe{Int64}

In [44]:
typeof(MyTypeAndMe(17.0))

MyTypeAndMe{Float64}

In [45]:
typeof(MyTypeAndMe(Float16(17)))

MyTypeAndMe{Float16}

In [46]:
typeof(MyTypeAndMe("17"))

MyTypeAndMe{String}

One can read the doc on <a href="https://docs.julialang.org/en/v1/manual/types/#Parametric-Types-1"> parametric types, </a> but warning: you may find it a little confusing. (I did!)  <br>

Maybe best to understand that `NumAndType(x)` creates an object whose type is `NumAndType(`typeof(x)`)` and whose "n field" is `x`.

In [47]:
s = MyTypeAndMe(42)

MyTypeAndMe{Int64}(42)

In [48]:
s.n

42

In [49]:
dump(s)

MyTypeAndMe{Int64}
  n: Int64 42


In [50]:
typeof(s)

MyTypeAndMe{Int64}

8.  Explain s.n, dump(s), and typeof(s) for the following.

**Explanation**: s.n will have type T = Array{Float64,1} since we fed it with rand(5) which returns an Array{Float64,1} by default, which also explains why we're getting the five Float64 elements of s.n through dumps(s) and why typeof(s) returns MyTypeAndMe{Array{Float64,1}}.

In [51]:
s = MyTypeAndMe(rand(5))

MyTypeAndMe{Array{Float64,1}}([0.460352, 0.27565, 0.561169, 0.391871, 0.451187])

In [52]:
s.n

5-element Array{Float64,1}:
 0.4603516556245466
 0.2756498245363379
 0.5611694340814712
 0.3918707849260812
 0.4511867983088462

In [53]:
dump(s)

MyTypeAndMe{Array{Float64,1}}
  n: Array{Float64}((5,)) [0.460352, 0.27565, 0.561169, 0.391871, 0.451187]


In [54]:
typeof(s)

MyTypeAndMe{Array{Float64,1}}

# A Use Case for Parameterized Types

In mathematics there is the concept of field extensions such as $\mathbb{Q}[\sqrt{2}]$ which means arithmetic operations with $a+b\sqrt{2}$ where $a$ an $b$ are rational. One can also talk about $\mathbb{Z}[\sqrt{2}]$ where
one extends the integers allowing only plus, minus, and multiply perhaps. Let's make this general in Julia:

In [55]:
struct ExtendSqrt2{T}
    a::T
    b::T
end

In [56]:
Base.:show(io::IO,  x::ExtendSqrt2) = print(io, "$(x.a)+$(x.b)√2")

In [57]:
ExtendSqrt2(3,4)

3+4√2

In [58]:
typeof(ExtendSqrt2(3,4))

ExtendSqrt2{Int64}

In [59]:
ExtendSqrt2(3.5,4.1)

3.5+4.1√2

In [60]:
typeof(ExtendSqrt2(3.5,4.1))

ExtendSqrt2{Float64}

In [61]:
ExtendSqrt2(1//3,1//2)

1//3+1//2√2

In [62]:
typeof(ExtendSqrt2(1//3,1//2))

ExtendSqrt2{Rational{Int64}}

9. Extend +,-,* by defining Base.:+ etc. Demonstrate that these work.   <br>

10. Create a matrix whose elements are ExtendSqrt2 and an ExtendSqrt2 consisting of two matrices.  What are the two different types?  What are the operations?

**Answer**: As expected, a matrix whose elements are ExtendSqrt2 has type Array{ExtendSqrt2{Int64},2}, and an ExtendSqrt2 consisting of two matrices has the "inverse" type ExtendSqrt2{Array{Int64,2}}.

In [63]:
Base.:*(x::ExtendSqrt2, y::ExtendSqrt2) = ExtendSqrt2(x.a*y.a + 2x.b*y.b, x.a*y.b + y.a*x.b)
Base.:+(x::ExtendSqrt2, y::ExtendSqrt2) = ExtendSqrt2(x.a + y.a, x.b + y.b)
Base.:-(x::ExtendSqrt2, y::ExtendSqrt2) = ExtendSqrt2(x.a - y.a, x.b - y.b)

In [65]:
[ExtendSqrt2(i,j) for i=1:3,j=1:3]^2  # Demonstrates that * works!

3×3 Array{ExtendSqrt2{Int64},2}:
 18+17√2  30+20√2  42+23√2
 24+20√2  36+26√2  48+32√2
 30+23√2  42+32√2  54+41√2

In [67]:
ExtendSqrt2(2,4) + ExtendSqrt2(-5, 5)  # Demonstrates that + works.

-3+9√2

In [71]:
ExtendSqrt2(10,3) - ExtendSqrt2(-2,-1)  # Demonstrates that - works.

12+4√2

In [72]:
typeof([ExtendSqrt2(i,j) for i=1:3,j=1:3])

Array{ExtendSqrt2{Int64},2}

In [76]:
typeof(ExtendSqrt2([1 2; 3 4], [5 6; 7 8]))

ExtendSqrt2{Array{Int64,2}}

# Another Julia type is a symbol

In [77]:
s = :abc

:abc

In [78]:
typeof(s)

Symbol

In [79]:
MeAndMyType(:abc)

UndefVarError: UndefVarError: MeAndMyType not defined

# Creatively using these ideas.

Today's scientific computing and machine learning needs computational graphs for automatic differentiation, optimization and so many more uses. See if this makes sense to you.  We will get back to this later in the semester.

In [80]:
abstract type HW2 end # This creates an abstract type called HW2 so we don't mix things up

If you wish to read about <a href=https://docs.julialang.org/en/v1/manual/types/#Abstract-Types-1> Abstract Types </a> .  Don't worry if this doesn't fully make sense yet.

In [81]:
struct Plus{A, B} <: HW2
    a::A
    b::B
end

In [82]:
Base.:+(x::Symbol, y::Symbol) = Plus(x,y)
Base.:+(x::HW2, y::Symbol) = Plus(x,y)

In [87]:
:a + :b

Plus{Symbol,Symbol}(:a, :b)

In [84]:
:a + :b + :c

Plus{Plus{Symbol,Symbol},Symbol}(Plus{Symbol,Symbol}(:a, :b), :c)

In [85]:
:a + :b + :c + :d

Plus{Plus{Plus{Symbol,Symbol},Symbol},Symbol}(Plus{Plus{Symbol,Symbol},Symbol}(Plus{Symbol,Symbol}(:a, :b), :c), :d)

11. Explain the types and values of the above summations of symbols. Explain how this could be used in a computational graph.

**Answer**: :a + :b is of type Plus{Symbol, Symbol} and could be represented by a tree with Plus at the root and :a, :b at the leaves. The types get recursively long pretty quickly but could be stored as a tree that can be traversed recursively to perform some calculation (on numbers or symbols) or execute some expression. I suppose this foreshadows the upcoming symbolic calculator.

# A mini-symbolic calculator

We are not asking you to anything here but to show you how one can begin building a full symbolic calculator with just a screenful of lines in Julia. By the end of the course you will learn how it all works.

In [52]:
abstract type Op end

struct Add{A, B}  <: Op
    a::A
    b::B
end

struct Subtract{A, B}  <: Op
    a::A
    b::B
end

struct Max{A, B}  <: Op
    a::A
    b::B
end

struct Mul{A,B}  <: Op
    a::A
    b::B
end

struct LazyVar <: Op
   s :: Symbol
end

Base.:show(io::IO, format::MIME"text/html", x::LazyVar) = print(io,"<b>", x.s, "</b>")

macro var(v)    
   esc(:($v = $(LazyVar(v))))
end

function evaluate(x::Add; vals...) 
      evaluate(x.a; vals...) + evaluate(x.b; vals...)
end
Base.:+(x::Op, y::Op) = Add(x,y)
Base.:+(x::Op, y::Number) = Add(x,y)
Base.:+(x::Number, y::Op) = Add(x,y)


#sub
function evaluate(x::Subtract; vals...) 
      evaluate(x.a; vals...) - evaluate(x.b; vals...)
end
Base.:-(x::Op, y::Op) = Subtract(x,y)

#max
function evaluate(x::Max; vals...) 
     max(evaluate(x.a; vals...), evaluate(x.b; vals...))
end
Base.:max(x::Op, y::Op) = Max(x,y)
   

#mul
function evaluate(x::Mul; vals...) 
      evaluate(x.a; vals...) * evaluate(x.b; vals...)
end
Base.:*(x::Op, y::Op ) = Mul(x,y)
Base.:*(x::Number,y::Op) = Mul(x,y)
Base.:*(x::Op,y::Number) = Mul(x,y)

evaluate(x::LazyVar; vals...) = vals.data[x.s]  
evaluate(x; vals...) = x

evaluate (generic function with 6 methods)

In [53]:
@var x

LazyVar(:x)

In [54]:
@var y

LazyVar(:y)

In [55]:
u = x + 3*y
v = max(u,10*y)
w = y*x*u

Mul{Mul{LazyVar,LazyVar},Add{LazyVar,Mul{Int64,LazyVar}}}(Mul{LazyVar,LazyVar}(LazyVar(:y), LazyVar(:x)), Add{LazyVar,Mul{Int64,LazyVar}}(LazyVar(:x), Mul{Int64,LazyVar}(3, LazyVar(:y))))

In [56]:
for t∈[u,v,w]
 println(evaluate(t,x=5,y=4))
end

17
40
340
